# 1. 初期設定

In [ ]:
#@markdown はじめに、サイトの情報を入力して、このセルを実行してください。<br>初期設定には環境によっては時間がかかる場合があります。ブラウザを閉じずにそのままお待ちください。
site_title = "\u3053\u3053\u306B\u30BF\u30A4\u30C8\u30EB\u3092\u5165\u529B" #@param {type:"string"}
site_author = "\u3053\u3053\u306B\u30B5\u30A4\u30C8\u306E\u4F5C\u6210\u8005\u3092\u5165\u529B" #@param {type:"string"}

##################
# 変数の初期設定 #
##################

site_document_root = "/"

_dakit_site_title = site_title
_dakit_site_title_use_logo = False
_dakit_site_author = site_author
_dakit_site_noindex = True
_dakit_site_document_root = site_document_root

# site_document_rootに続く形で、最初のスラッシュは付けずに書く
_dakit_site_index_loc = "index.html"
_dakit_site_js_loc = "assets/dakit.js"
#_dakit_logo_loc = "assets/logo.svg"
#_dakit_logo_height = 24
#_dakit_logo_custom_style = "margin-top:-0.4rem"
_dakit_site_catalog_loc = "data.tsv"

_dakit_list_item_id_col_name = "item_id"
_dakit_list_item_title_col_name = ""
_dakit_list_item_url_col_name = "item_url"
_dakit_list_item_image_col_name = "item_image"

_dakit_theme_css_from_cdn = True
_dakit_theme_css_cdn_tag = '''
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
'''
#_dakit_theme_css_loc = "assets/bootstrap.min.css"
_dakit_theme_css_page_custom_css = '''
<style>body {min-height: 75rem; padding-top: 4.5rem;}</style>
'''

_dakit_theme_js_from_cdn = True
_dakit_theme_js_cdn_tag = '''
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous"></script>
'''
#_dakit_theme_js_loc = "assets/bootstrap.min.js"

_dakit_navbar_color = "navbar-dark bg-dark"

dakit_config = {}
_cfg_site_logo   = {"enabled": _dakit_site_title_use_logo}
_cfg_site_list   = {"item_id_col_name": _dakit_list_item_id_col_name, "item_title_col_name": _dakit_list_item_title_col_name, "item_url_col_name": _dakit_list_item_url_col_name, "item_image_col_name": _dakit_list_item_image_col_name}
_cfg_site_navbar = {"expand": True, "color": _dakit_navbar_color}
_cfg_site_theme  = {"css_from_cdn": _dakit_theme_css_from_cdn, "css_cdn_tag": _dakit_theme_css_cdn_tag, "js_from_cdn": _dakit_theme_js_from_cdn, "js_cdn_tag": _dakit_theme_js_cdn_tag, "page_custom_css": _dakit_theme_css_page_custom_css}

dakit_config["site"] = {
    "title": _dakit_site_title,
    "author": _dakit_site_author,
    "noindex": _dakit_site_noindex,
    "index_loc": _dakit_site_index_loc,
    "js_loc" : _dakit_site_js_loc,
    "document_root": _dakit_site_document_root,
    "catalog_loc": _dakit_site_catalog_loc,
    "logo": _cfg_site_logo, "list": _cfg_site_list, "navbar": _cfg_site_navbar, "theme": _cfg_site_theme}

######################
# モジュール読み込み #
######################

!apt-get install libmagic1 >/dev/null
!apt-get install tidy >/dev/null

!pip install Pillow >/dev/null
!pip install pandas >/dev/null
!pip install bs4 >/dev/null
!pip install jinja2 >/dev/null
!pip install kora >/dev/null
!pip install python-magic >/dev/null
!pip install pytidylib >/dev/null
!pip install tqdm >/dev/null

import csv
import datetime
from io import BytesIO
import json
import os
import re
import sys
import time
import urllib.parse

import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import drive, files
from IPython.display import clear_output, display, HTML
from jinja2 import Environment, FileSystemLoader, Template
from kora.xattr import get_id
import magic
from PIL import Image, ImageOps
from tidylib import tidy_document
from tqdm.notebook import tqdm

%cd "/content/"
!rm -rf dakit
!git clone https://github.com/utokyodh/dakit

%cd "dakit"
!rm -rf output

ENV = Environment(
    loader=FileSystemLoader("./templates/"),
    trim_blocks=True,
    lstrip_blocks=True
)

def upload_file_in_colab() -> str:
    uploaded_file = files._upload_files(multiple=False)

    if not uploaded_file:
        return

    if len(uploaded_file) > 1:
        raise ValueError("このバージョンのノートブックでは、一度に読み込めるファイルは1つだけです。")

    filename, data = list(uploaded_file.items())[0]
    filename = files._get_unique_filename(filename)

    with open(filename, "wb") as f:
        f.write(data)

    return filename

def dakit_load_spreadsheet(filename):

    mime_type = magic.from_file(filename, mime=True)
    
    if mime_type == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet" \
    or mime_type == "application/vnd.ms-excel":
        df = pd.read_excel(filename, dtype="str").fillna("")
    elif mime_type == "text/plain":
        if magic.from_file(filename, mime=False) not in ["UTF-8 Unicode text", "ASCII text"]:
            raise ValueError("CSV・TSVファイルの文字コードはUTF-8である必要があります。")
        else:
            with open(filename, encoding="utf-8") as f1:
                td1024 = f1.read(1024)
                f1.seek(0)
                sniffer = csv.Sniffer()
                delimiter = sniffer.sniff(td1024).delimiter
                has_header = sniffer.has_header(td1024)
            if delimiter not in ["\t", ","]:
                raise ValueError("区切り文字が読み取れませんでした。有効なCSV・TSVファイルではないようです。")
            else:
                if delimiter == ",":
                    df = pd.read_csv(filename, header=0, dtype="str").fillna("")
                else:
                    df = pd.read_table(filename, header=0, dtype="str").fillna("")
                if not has_header:
                    print("[Info]ヘッダーが存在する確証が得られませんでした。先頭行が列名として使用されます。")
    else:
        raise ValueError("対応していないファイル形式です。")

    return df

def dakit_validate_catalog_df(df):

    columns = df.columns.values

    if columns[0] != dakit_config["site"]["list"]["item_id_col_name"]:
        raise ValueError(f"""表の一番左の列は'{dakit_config["site"]["list"]["item_id_col_name"]}'という名称である必要があります。""")
    elif columns[1] != dakit_config["site"]["list"]["item_image_col_name"]:
        raise ValueError(
            f"""表の左から2番目の列は'{dakit_config["site"]["list"]["item_image_col_name"]}'という名称である必要があります。""")
    elif len(columns) != len(set(columns)):
        raise ValueError("列名は重複しないように指定してください。")

    item_id_list = df[dakit_config["site"]["list"]["item_id_col_name"]].to_list()

    if len(item_id_list) != len(set(item_id_list)):
        raise ValueError(f"""{dakit_config["site"]["list"]["item_id_col_name"]}は重複しないように指定してください。""")

    return df

print("\N{WHITE HEAVY CHECK MARK}初期設定が完了しました")

In [ ]:
#@markdown このセルを実行するとタイトル部分の色を変更することができます。実行するとプレビューが表示されます。
navbar_color_theme = "navbar-dark bg-dark" #@param ["navbar-dark bg-primary", "navbar-dark bg-dark", "navbar-dark bg-danger", "navbar-light bg-secondary"]
dakit_config["site"]["navbar"]["color"] = navbar_color_theme

preview_htm = HTML(f'''
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
</head>
<body>
    <div class="container" style="height: 100px">
        <nav class="navbar fixed-top navbar-expand-sm {dakit_config["site"]["navbar"]["color"]}">
            <div class="container">
                <a class="navbar-brand" href="#">{ dakit_config['site']['title'] }</a>
                <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#dakit-navbar"
                    aria-controls="dakit-navbar" aria-expanded="false" aria-label="Toggle navigation">
                    <span class="navbar-toggler-icon"></span>
                </button>
            </div>
        </nav>
    </div>
</body>
''')

display(preview_htm)

# 2. データのアップロード

アップロードできるファイルは以下の通りです。

* *.tsv（タブ区切りテキスト）
* *.csv（カンマ区切りテキスト）
* *.xlsx（Excel ブック）
* *.xls（Excel 97-2004 ブック）

Excel ブックの場合、ファイルの最初のワークシートだけが読み込まれます。文字コードはUTF-8を推奨します。


In [ ]:
#@markdown このセルを実行して、資料データ（catalog）をアップロードしてください。
catalog_filename = upload_file_in_colab()
dakit_catalog = dakit_validate_catalog_df(dakit_load_spreadsheet(catalog_filename))
clear_output()
print("\N{WHITE HEAVY CHECK MARK}資料データのアップロードが完了しました。データの列名は以下の通りです。")
print(dakit_catalog.columns.values)

# 3. 資料データの設定

In [ ]:
#@markdown 資料の各アイテムの表示名（Webページ上に表示されるタイトル）が入力されている列名を入力し、このセルを実行してください。
item_title_column_name = "\u3053\u3053\u306B\u5217\u540D\u3092\u5165\u529B" #@param {type:"string"}
if item_title_column_name in dakit_catalog.columns.values:
    dakit_config["site"]["list"]["item_title_col_name"] = item_title_column_name
    print(f"\N{WHITE HEAVY CHECK MARK}列「{item_title_column_name}」を表示名に指定しました。")
else:
    msg = "指定された列名が資料データに存在しませんでした。列名は次の中から指定する必要があります：\n" + str(dakit_catalog.columns.values)
    raise ValueError(msg)

# 4. 検索ページの生成



In [ ]:
#@markdown このセルを実行して、検索ページを生成してください。
os.makedirs("./output/assets", exist_ok=True)
list_page_template = ENV.get_template("list.jinja")
js_template = ENV.get_template("dakit-minimal-single.js.jinja")

data = {"dakit_config": dakit_config}

with open("./output/index.html", mode="w") as f:
    document, errors = tidy_document(list_page_template.render(data), options={"numeric-entities": 1, "indent": "auto"})
    f.write(document)

with open("./output/assets/dakit.js", mode="w") as f:
    f.write(js_template.render(data))

print("\N{WHITE HEAVY CHECK MARK}生成が完了しました。")

# 5. 個別ページの生成

In [ ]:
#@markdown このセルを実行して、アイテムごとの個別ページを生成してください。
dakit_catalog_keys = [ x for x in list(dakit_catalog.columns.values) if x not in [dakit_config["site"]["list"]["item_id_col_name"], dakit_config["site"]["list"]["item_image_col_name"]]]

dakit_has_pdf_image = False

for item in dakit_catalog.iterrows():
    dic = item[1].to_dict()
    for k in dakit_catalog_keys:
        dic[k] = str(dic[k])
    template = ENV.get_template("item-single.jinja")
    page_config = {}
    page_config["page_title"] = dic[dakit_config["site"]["list"]["item_title_col_name"]] + " - " + dakit_config["site"]["title"]
    page_config["item_title"] = dic[dakit_config["site"]["list"]["item_title_col_name"]]
    
    image_name = dic[dakit_config["site"]["list"]["item_image_col_name"]]
    
    if image_name == "":
        page_config["image_type"] = ""
        page_config['image_single_loc'] = ""
    elif image_name.split(".")[-1].lower() == "pdf":
        dakit_has_pdf_image = True
        page_config["image_type"] = "pdf"
        page_config['image_single_loc'] = dic[dakit_config["site"]["list"]["item_image_col_name"]]
    else:
        page_config["image_type"] = "single"
        page_config['image_single_loc'] = dic[dakit_config["site"]["list"]["item_image_col_name"]]
    
    data = {"dakit_config": dakit_config, "page_config": page_config, "dic":dic, "keys":dakit_catalog_keys}
    with open(f"""./output/{dic[dakit_config["site"]["list"]["item_id_col_name"]]}.html""", mode="w") as f:
        document, errors = tidy_document(template.render(data), options={"numeric-entities": 1, "indent": "auto"})
        f.write(document)
        
if dakit_has_pdf_image:
    print("PDFファイルを表示するために、次のセルを実行して必要なファイルをダウンロードしてください。")    

print("\N{WHITE HEAVY CHECK MARK}生成が完了しました。")

In [ ]:
#@markdown PDFファイルを公開する場合は、このセルを実行して追加のファイルをダウンロードします。
!wget https://github.com/mozilla/pdf.js/releases/download/v3.1.81/pdfjs-3.1.81-dist.zip
!unzip pdfjs-3.1.81-dist.zip -d ./output/pdfjs

# 6. 検索ページ用TSVファイルの作成

In [ ]:
#@markdown このセルを実行して、検索ページ用のTSVファイルを生成してください。

dakit_catalog[dakit_config["site"]["list"]["item_url_col_name"]] = dakit_config["site"]["document_root"] + dakit_catalog[dakit_config["site"]["list"]["item_id_col_name"]] + ".html"
dakit_catalog.to_csv("./output/data.tsv", sep="\t", index=False)

print("\N{WHITE HEAVY CHECK MARK}生成が完了しました。")

# 7. ダウンロード

In [ ]:
#@markdown このセルを実行して、完成したファイル一式をzipファイルとしてダウンロードしてください。
!zip -r output.zip output/
files.download("output.zip")